In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings

import pandas as pd
import numpy as np
import os
import time
import pickle
import json

reversechain_apis = "/home/cc/AutomaticWorkflowGeneration/ActionEngine/db_new/api_info_new/reversechain_apis.json"
df = pd.read_json(reversechain_apis)
df.head()

,name,Description,input_params,output_params,format
0,Username2ID,This API is to convert a username to a user ID.,{'user_name': {'description': 'the name of the...,{'user_ID': {'description': 'the ID of the use...,Username2ID(user_name) -> user_ID
1,Playlistname2ID,This API is to convert a playlist name to a pl...,{'playlist_name': {'description': 'the name of...,{'playlist_ID': {'description': 'the ID of the...,Playlistname2ID(playlist_name) -> playlist_ID
2,AddSongToPlaylist,This API is to add a song to a playlist.,{'user_ID': {'description': 'the ID of the use...,{'playlist_songs': {'description': 'a list of ...,"AddSongToPlaylist(user_ID, playlist_ID, song_n..."
3,GeoLocation2TimeZone,This API is to convert geolocation to timezone.,{'geolocation': {'description': 'the geolocati...,{'timezone': {'description': 'the timezone of ...,GeoLocation2TimeZone(geolocation) -> timezone
4,SetAlarm,This API allows the user to set an alarm in a ...,{'timezone': {'description': 'the timezone whe...,{'alarm_status': {'description': 'the status o...,"SetAlarm(timezone, time) -> alarm_status"


# Chack the eval_api_info

In [ ]:
len(eval_api_df)

88

In [3]:
len(eval_api_df)
categories = {}
for i in range(len(eval_api_df)):
    if eval_api_df["category"][i] not in categories:
        categories[eval_api_df["category"][i]] = 1
        print(eval_api_df["category"][i])
    else:
        categories[eval_api_df["category"][i]] += 1
print("---------------------------------------------")
print("Number of Categories:" ,len(categories))


Jobs
Music
Travel
Other
Entertainment
Gaming
Database
Business_Software
Artificial_Intelligence_Machine_Learning
Monitoring
SMS
Commerce
Health_and_Fitness
News_Media
Data
Events
Media
Medical
Email
Financial
Payments
Communication
Location
Finance
Advertising
Cybersecurity
Science
Social
Movies
Tools
eCommerce
Business
Transportation
Weather
Visual_Recognition
Video_Images
Education
Translation
Text_Analysis
Mapping
Cryptography
Search
Logistics
Food
Storage
Sports
Energy
---------------------------------------------
Number of Categories: 47


In [4]:
with open('./count_by_category.json', 'w') as json_file:
    json.dump(categories, json_file, indent=4)

FileNotFoundError: [Errno 2] No such file or directory: '/home/cc/AutomaticWorkflowGeneration/ActionEngine/db/api_info_new/reversechain_apis.json'

# Merge original funciton info and eval api info

In [57]:
"""
Check
"""
for c in eval_api_df.columns:

    print(f'{c:<30}: {eval_api_df.loc[0, c]}')
print("Number of Categories:" ,len(eval_api_df))
print('+'*40)

for c in api_df.columns:

    print(f'{c:<30}: {api_df.loc[0, c]}')
print("Number of Categories:" ,len(api_df))

api_name                      : Random Gradient Resume Template Generator
category                      : Jobs
summary                       : The API generates unique resume templates with randomly generated personal, skill, and experience data using the Flask web framework and the Faker library. Users can access the main endpoint for random template generation via a GET method, which renders an HTML page displaying the created resume template. The generated resumes include a name, job title, website, skills, summary, and work experiences based on realistic information. Additionally, users can download the generated template as a ZIP file through the download endpoint. Both endpoints have no required or optional parameters. The output of the main endpoint is an HTML page displaying the random resume template, while the download endpoint generates a ZIP file containing the template and its assets.
name                          : nan
Number of Categories: 179
+++++++++++++++++++++++++++

In [5]:
"""
Marge the 2 dataframes: eval_api_df(179) + api_df(15) => combined_df (194)
"""

api_df["category"] = "Image Generation"
# Step 1: Select and rename columns in eval_api_df to match api_df
eval_api_df_selected = eval_api_df[['api_name', 'category', 'summary']]
eval_api_df_selected.rename(columns={'api_name': 'name'}, inplace=True)

# Step 2: Add missing columns to eval_api_df_selected (fill with NaN or default values)
columns_to_add = [col for col in api_df.columns if col not in eval_api_df_selected.columns]
for col in columns_to_add:
    eval_api_df_selected[col] = None  # or fill with a default value

# Step 3: Align columns and concatenate vertically
eval_api_df_selected = eval_api_df_selected[api_df.columns]  # Ensure column order matches
df = pd.concat([api_df, eval_api_df_selected], axis=0, ignore_index=True)
df['Id'] = range(1, len(df) + 1)

df = df.filter(items=['Id','name', 'category', 'summary'])

print("Number of items: ",len(df))


Number of items:  194


/tmp/ipykernel_13200/2252715759.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_api_df_selected.rename(columns={'api_name': 'name'}, inplace=True)


In [6]:
"""
Check
"""
print(df.columns)
category_counts = df['category'].unique()
print(category_counts)
print(len(category_counts))
df.tail()

Index(['Id', 'name', 'category', 'summary'], dtype='object')
['Image Generation' 'Jobs' 'Music' 'Travel' 'Other' 'Entertainment'
 'Gaming' 'Database' 'Business_Software'
 'Artificial_Intelligence_Machine_Learning' 'Monitoring' 'SMS' 'Commerce'
 'Health_and_Fitness' 'News_Media' 'Data' 'Events' 'Media' 'Medical'
 'Email' 'Financial' 'Payments' 'Communication' 'Location' 'Finance'
 'Advertising' 'Cybersecurity' 'Science' 'Social' 'Movies' 'Tools'
 'eCommerce' 'Business' 'Transportation' 'Weather' 'Visual_Recognition'
 'Video_Images' 'Education' 'Translation' 'Text_Analysis' 'Mapping'
 'Cryptography' 'Search' 'Logistics' 'Food' 'Storage' 'Sports' 'Energy']
48


,Id,name,category,summary
189,190,Face Detection with Age and Gender Estimation,Video_Images,The Face Detection with Age and Gender Estimat...
190,191,Hotels,Travel,The Hotels API provides comprehensive informat...
191,192,Call of Duty,Gaming,The Call of Duty API provides access to stats ...
192,193,Funny Joke Dataset,Database,The Funny Joke Dataset API provides a collecti...
193,194,BasketAPI,Sports,BasketAPI provides comprehensive basketball da...


In [91]:
df['Id'][0]

1

## Divide by Number of APIs

In [127]:
# List of row counts where you want to split the DataFrame
row_splits = [20, 40, 60, 80, 100, 120, 140, 160, 180, 194]

# Create empty list to store the resulting DataFrames
dfs_numbers = []

# Initialize the start index
start_index = 0

# Loop over the row splits to create separate DataFrames
for end_index in row_splits:
    # Select the rows for the current split
    split_df = df.iloc[:end_index]
    dfs_numbers.append(split_df)
    
    # Update the start index for the next split
    start_index = end_index

In [128]:
len(dfs_numbers)

10

## Divide by Number of Domains

In [129]:
categories = [
    'Image Generation', 'Jobs', 'Music', 'Travel', 'Other', 'Entertainment',
    'Gaming', 'Database', 'Business_Software',
    'Artificial_Intelligence_Machine_Learning', 'Monitoring', 'SMS', 'Commerce',
    'Health_and_Fitness', 'News_Media', 'Data', 'Events', 'Media', 'Medical',
    'Email', 'Financial', 'Payments', 'Communication', 'Location', 'Finance',
    'Advertising', 'Cybersecurity', 'Science', 'Social', 'Movies', 'Tools',
    'eCommerce', 'Business', 'Transportation', 'Weather', 'Visual_Recognition',
    'Video_Images', 'Education', 'Translation', 'Text_Analysis', 'Mapping',
    'Cryptography', 'Search', 'Logistics', 'Food', 'Storage', 'Sports', 'Energy'
]

# List of row counts where you want to split the DataFrame
row_splits = [5, 10,15, 20, 25, 30, 35, 40, 48]

# Create empty list to store the resulting DataFrames
dfs_domain = []

# Initialize the start index
start_index = 0

# Loop over the row splits to create separate DataFrames
for end_index in row_splits:
    # Select the rows for the current split
    categories_list = categories[:end_index]
    split_df = df[df['category'].isin(categories_list)]
    dfs_domain.append(split_df)
    
    # Update the start index for the next split
    start_index = end_index

In [130]:
print("Number of Split", len(dfs_domain))
print(len(dfs_domain[-1]))

Number of Split 9
194


# Make differrent Dataframes

In [131]:
df.columns

Index(['Id', 'name', 'category', 'summary'], dtype='object')

# Create Vector Databese

In [141]:
"""
Generate vector database for different number of APIs 
[20, 40, 60, 80, 100, 120, 140, 160, 180, 194]
"""
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

for i in range(len(dfs_numbers)):
    passage_data = dfs_numbers[i]
    start=time.time()
    metadatas = []
    for index, row in passage_data.iterrows():
        doc_meta = {
            "id": row['Id'],
            "name": row['name'],
            "category": row['category']
        }
        metadatas.append(doc_meta)

    faiss = FAISS.from_texts(passage_data['summary'].tolist(), embedding_function, metadatas)
    print("Time Taken --> ", time.time()-start) 
    num = len(passage_data)
    faiss.save_local(path + 'vectordb/LangChain_FAISS/eval_numbers', f"eval_api_vec_num_{num}")

Time Taken -->  0.027750015258789062
Time Taken -->  0.055271148681640625
Time Taken -->  0.06352424621582031
Time Taken -->  0.07059979438781738
Time Taken -->  0.08420610427856445
Time Taken -->  0.11052083969116211
Time Taken -->  0.13030004501342773
Time Taken -->  0.1194465160369873
Time Taken -->  0.1652538776397705
Time Taken -->  0.16869497299194336


In [142]:
"""
Generate vector database for different number of APIs 
[5, 10,15, 20, 25, 30, 35, 40, 48]
"""
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

split = [5, 10, 15, 20, 25, 30, 35, 40, 48]
for i in range(len(dfs_domain)):
    passage_data = dfs_domain[i]
    start=time.time()
    metadatas = []
    for index, row in passage_data.iterrows():
        doc_meta = {
            "id": row['Id'],
            "name": row['name'],
            "category": row['category']
        }
        metadatas.append(doc_meta)

    faiss = FAISS.from_texts(passage_data['summary'].tolist(), embedding_function, metadatas)
    print("Time Taken --> ", time.time()-start) 
    num = str(split[i])
    faiss.save_local(path + 'vectordb/LangChain_FAISS/eval_domains', f"eval_api_vec_domain_{num}")

Time Taken -->  0.032562971115112305
Time Taken -->  0.04890608787536621
Time Taken -->  0.0625
Time Taken -->  0.08231306076049805
Time Taken -->  0.0986180305480957
Time Taken -->  0.11283135414123535
Time Taken -->  0.11978411674499512
Time Taken -->  0.12639284133911133
Time Taken -->  0.14655709266662598


## Try Vector DB

In [40]:
loaded_faiss = FAISS.load_local(path + 'vectordb/LangChain_FAISS/', embedding_function, "api_vec", allow_dangerous_deserialization=True)
k = 16
start=time.time()
Top_k = loaded_faiss.similarity_search_with_score("Animated image Generation?",k)

In [42]:

print(Top_k[15])

IndexError: list index out of range

In [19]:
for i in range(len(Top_k))
Top_k[15][0].metadata["id"]

14